In [17]:
import os
import cv2
from methods.local_feature.orb import run
import numpy as np
from types import SimpleNamespace  # For mock configuration
from third_party.colmap.scripts.python.read_write_model import qvec2rotmat
from utils.path_helper import get_fullpath_list
import matplotlib.pyplot as plt

In [18]:
def convert_to_cv2_keypoints(keypoints):
    return [cv2.KeyPoint(x=kp[0], y=kp[1], _size=1) for kp in keypoints]

In [19]:
def draw_keypoints_on_image(image, keypoints):
    # Convert keypoints to OpenCV KeyPoint objects
    opencv_keypoints = convert_to_cv2_keypoints(keypoints)

    # Draw keypoints on the image
    output_image = cv2.drawKeypoints(image, opencv_keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    return output_image

In [20]:
# Configuration for keypoint extraction
cfg = SimpleNamespace()
cfg.method_dict = {
    'config_common': {
        'num_keypoints': 500,
        'keypoint': 'orb-def'
    }
}

# Directories and file paths
data_dir = "../data/phototourism/piazza_san_marco"
output_dir = "../data/phototourism/piazza_san_marco/output/orb"
os.makedirs(output_dir, exist_ok=True)

image_list_path = os.path.join(data_dir, "images.txt")
if not os.path.exists(image_list_path):
    raise FileNotFoundError(f"{image_list_path} not found.")

# Load image paths
with open(image_list_path, "r") as f:
    image_list = [os.path.normpath(os.path.join(data_dir, "images", line.strip())) for line in f]

print(f"Loaded {len(image_list)} images.")

# Extract keypoints and descriptors for all images
features = {}
for i, image_path in enumerate(image_list):
    features[i] = run(image_path, cfg)
    print(f"Image {i + 1}: {len(features[i]['kp'])} keypoints detected.")

    # Save keypoints visualization for each image
    image = cv2.imread(image_path)
    image_with_keypoints = draw_keypoints_on_image(image, features[i]['kp'])
    keypoints_output_path = os.path.join(output_dir, f"image_{i + 1}_keypoints.jpg")
    cv2.imwrite(keypoints_output_path, image_with_keypoints)

print("Keypoints detection completed and saved.")

# Perform matching for every pair of images
num_images = len(image_list)
matches_results = []

for i in range(num_images // 2):
    # Pair indices
    idx1 = i * 2
    idx2 = idx1 + 1

    # Descriptors
    desc1 = np.array(features[idx1]['descs'], dtype=np.uint8)
    desc2 = np.array(features[idx2]['descs'], dtype=np.uint8)

    # FLANN Matcher
    index_params = dict(algorithm=6, table_number=6, key_size=12, multi_probe_level=2)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desc1, desc2, k=2)

    # Separate matches into good and bad
    good_matches = []
    bad_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
        else:
            bad_matches.append(m)

    # RANSAC-based geometric verification
    if len(good_matches) < 8:  # 8 points is required for fundamental matrix estimation
        print(f"Insufficient matches for pair {i + 1}. Skipping.")
        continue

    pts1 = np.float32([features[idx1]['kp'][m.queryIdx] for m in good_matches])
    pts2 = np.float32([features[idx2]['kp'][m.trainIdx] for m in good_matches])
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, 1.0, 0.99)

    if F is None or mask is None:
        print(f"RANSAC failed for pair {i + 1}. Skipping.")
        continue
    
    # Save the fundamental matrix
    fundamental_matrix_txt_path = os.path.join(output_dir, f"pair_{i + 1}_fundamental_matrix.txt")
    np.savetxt(fundamental_matrix_txt_path, F, fmt='%.6f', header="Fundamental Matrix")
    print(f"Fundamental matrix for pair {i + 1} saved to {fundamental_matrix_txt_path}.")

    # Extract inliers
    inlier_matches = [good_matches[k] for k in range(len(good_matches)) if mask[k]]
    print(f"Pair {i + 1}: {len(inlier_matches)} inliers after RANSAC.")

    # Visualize matches
    image1 = cv2.imread(image_list[idx1])
    image2 = cv2.imread(image_list[idx2])

    # Custom visualization: Draw good matches in blue, bad matches in red
    img_matches = image1.copy()
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    combined_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    combined_img[:h1, :w1] = image1
    combined_img[:h2, w1:] = image2

    # Draw lines for matches
    for m in bad_matches:  # Bad matches in red
        pt1 = tuple(map(int, features[idx1]['kp'][m.queryIdx]))
        pt2 = tuple(map(int, features[idx2]['kp'][m.trainIdx] + np.array([w1, 0])))
        cv2.line(combined_img, pt1, pt2, (0, 0, 255), 1)

    for m in good_matches:  # Good matches in blue
        pt1 = tuple(map(int, features[idx1]['kp'][m.queryIdx]))
        pt2 = tuple(map(int, features[idx2]['kp'][m.trainIdx] + np.array([w1, 0])))
        cv2.line(combined_img, pt1, pt2, (255, 0, 0), 2)

    matches_output_path = os.path.join(output_dir, f"pair_{i + 1}_matches.jpg")
    cv2.imwrite(matches_output_path, combined_img)

    # Store the results
    matches_results.append({
        "pair_index": i + 1,
        "num_inliers": len(inlier_matches),
        "fundamental_matrix": F
    })


Loaded 100 images.
Attempting to load image from: ..\data\phototourism\piazza_san_marco\images\01688348_12300730234.jpg
Attempting to load image: ..\data\phototourism\piazza_san_marco\images\01688348_12300730234.jpg
Image 1: 500 keypoints detected.
Attempting to load image from: ..\data\phototourism\piazza_san_marco\images\02577600_6870111213.jpg
Attempting to load image: ..\data\phototourism\piazza_san_marco\images\02577600_6870111213.jpg
Image 2: 500 keypoints detected.
Attempting to load image from: ..\data\phototourism\piazza_san_marco\images\03449722_3316267248.jpg
Attempting to load image: ..\data\phototourism\piazza_san_marco\images\03449722_3316267248.jpg
Image 3: 500 keypoints detected.
Attempting to load image from: ..\data\phototourism\piazza_san_marco\images\03693106_384020886.jpg
Attempting to load image: ..\data\phototourism\piazza_san_marco\images\03693106_384020886.jpg
Image 4: 500 keypoints detected.
Attempting to load image from: ..\data\phototourism\piazza_san_marco\